# Cuaderno 15: Planificación de la producción (2)
# (Varios productos, capacidad de producción)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\ZZ}{\mathbb{Z}_+}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

Existen numerosas variantes del problema de planificación de la producción (*lot sizing*). Presentamos aquí una variante donde se planifica la producción de dos bienes, tomando en cuenta la capacidad de producción instalada.

Como ejemplo, consideremos la planificación de los niveles óptimos de producción de una empresa dedicada a la producción de dos tipos de barras de cereales, sujeta a las siguientes condiciones:

* El plan de producción se organiza sobre un horizonte de tiempo de un año, dividido en un conjunto $T= \tabulatedset{1, \ldots, 12}$ de períodos mensuales. 
* Si en un período $t \in T$ se toma la decisión de producir (cualquiera de los dos productos), se incurre en un
  costo fijo igual a $c_t$ (en USD). 
* Denotaremos por $I:=\{1,2\}$ al conjunto que representa los dos tipos de barras de cereales a producir. Para cada período $t \in T$ y cada tipo de barra $i \in I$, se conocen la demanda estimada de cada en el mercado $d_{it}$ (en lotes de barras) y el costo unitario de su producción $p_{it}$ (en USD / lote). 
* En cada período se debe satisfacer toda la demanda estimada de cada tipo de barra. Si en un período el nivel de producción supera a la demanda, es posible almacenar el producto excedente a un costo unitario igual a $h_t$ (USD/lote). Este costo es el mismo para ambos tipos de barras. 
* La capacidad de producción de la empresa es constante y está limitada a un total de $C$ lotes por período, entre los dos tipos de barras.
* La producción se realiza por lotes. No es posible producir fracciones de un lote.
  
Se requiere determinar los niveles de producción mensuales para cada tipo de barras, para satisfacer toda la demanda al menor costo posible. Definimos las siguientes variables de decisión:

* variables enteras $x_{it}, \, i \in I, \, t \in T,$ que indican la cantidad de lotes del tipo $i$ a producir en el período $t$;
* variables binarias $y_t, \, t \in T,$ tales que $y_t = 1$ si y solamente si se toma la decisión producir al menos un tipo de barra en el período $t$;
* variables enteras $z_{it}, \, i \in I, \, t \in T,$ que indican la cantidad de lotes de barras del tipo $i$ a almacenar del período $t$ al período $t+1$.

Con estas definiciones, el problema puede ser formulado como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{t \in T} c_t y_t  + \sum_{i \in I} \sum_{t \in T} (p_{it} x_{it} + h_t z_{it})\\ 
& \mbox{s.r.}\\
& x_{i1} = d_{i1} + z_{i1}, \quad \forall i \in I, \\
& x_{it} + z_{i,t-1} = d_{it} + z_{it}, \quad \forall i \in I,  t \in T \setminus \{1\}, \\
& \sum_{i \in I} x_{it} \leq C y_t, \quad \forall t \in T, \\
& y_t \in \{0, 1\}, \quad \forall t \in T, \\
&x_{it}, z_{it} \in \ZZ, \quad \forall i \in I, t \in T.
\end{align*}


Vamos a implementar este programa utilizando la interfaz Python de Gurobi.

Definimos primero los conjuntos $I$ y $T$, y los parámetros $p$, $c$, $h$, $d$ y $C$:

In [2]:
from gurobipy import *

# Conjuntos y parametros del modelo
# Costos fijos y costos de almacenamiento por período:
T, c, h = multidict({
    1 : (400, 10),   2 : (450, 12),  3 : (400, 10),
    4 : (470,  9),   5 : (450, 12),  6 : (400, 10),
    7 : (350, 10),   8 : (400, 12),  9 : (450, 10),
   10 : (450, 12),  11 : (500, 15), 12 : (500,  5)})


# Costos de producción y demanda por (producto-período):
IxT, p, d = multidict({
    (1, 1) : (48, 20),   (1, 2) : (47, 20),  (1, 3) : (38, 25),
    (1, 4) : (45, 25),   (1, 5) : (47, 25),  (1, 6) : (42, 20),
    (1, 7) : (33, 20),   (1, 8) : (39, 18),  (1, 9) : (45, 18),
    (1,10) : (47, 20),   (1,11) : (50, 25),  (1,12) : (55, 27),
    (2, 1) : (44, 15),   (2, 2) : (42, 14),  (2, 3) : (35, 15),
    (2, 4) : (41, 16),   (2, 5) : (42, 15),  (2, 6) : (38, 17),
    (2, 7) : (33, 18),   (2, 8) : (36, 17),  (2, 9) : (41, 15),
    (2,10) : (42, 14),   (2,11) : (45, 15),  (2,12) : (47, 20)})

# Capacidad de la planta (lotes por período)
C = 40

# Recuperar conjunto I de productos desde IxT
I = tuplelist(set([i for (i,t) in IxT]))

# Definir conjunto T \ {1}
T2 = [t for t in T if not t==1]

print("T= {}".format(T))
print("I= {}".format(I))
print("T2= {}".format(T2))
print("c= {}".format(c))
print("h= {}".format(h))
print("p= {}".format(p))
print("d= {}".format(d))
print("C= {}".format(C))

T= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
I= [1, 2]
T2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
c= {1: 400, 2: 450, 3: 400, 4: 470, 5: 450, 6: 400, 7: 350, 8: 400, 9: 450, 10: 450, 11: 500, 12: 500}
h= {1: 10, 2: 12, 3: 10, 4: 9, 5: 12, 6: 10, 7: 10, 8: 12, 9: 10, 10: 12, 11: 15, 12: 5}
p= {(1, 1): 48, (1, 2): 47, (1, 3): 38, (1, 4): 45, (1, 5): 47, (1, 6): 42, (1, 7): 33, (1, 8): 39, (1, 9): 45, (1, 10): 47, (1, 11): 50, (1, 12): 55, (2, 1): 44, (2, 2): 42, (2, 3): 35, (2, 4): 41, (2, 5): 42, (2, 6): 38, (2, 7): 33, (2, 8): 36, (2, 9): 41, (2, 10): 42, (2, 11): 45, (2, 12): 47}
d= {(1, 1): 20, (1, 2): 20, (1, 3): 25, (1, 4): 25, (1, 5): 25, (1, 6): 20, (1, 7): 20, (1, 8): 18, (1, 9): 18, (1, 10): 20, (1, 11): 25, (1, 12): 27, (2, 1): 15, (2, 2): 14, (2, 3): 15, (2, 4): 16, (2, 5): 15, (2, 6): 17, (2, 7): 18, (2, 8): 17, (2, 9): 15, (2, 10): 14, (2, 11): 15, (2, 12): 20}
C= 40


Definimos ahora el objeto modelo y las variables del modelo:

In [3]:
m = Model('lot-sizing-2')

# lotes a producir
# x = m.addVars(I, T, vtype = GRB.INTEGER, name="x")
# alternativa:
x = m.addVars(IxT, vtype = GRB.INTEGER, name="x")

# se produce / no se produce en cada periodo
y = m.addVars(T, vtype = GRB.BINARY, name="y")

# lotes a almacenar
z = m.addVars(I, T, vtype = GRB.INTEGER, name="z")


--------------------------------------------
--------------------------------------------

Using license file /Users/ltorres/gurobi.lic
Academic license - for non-commercial use only


Construimos la función objetivo a partir de sus tres términos:

In [4]:
# costos variables de produccion
c_produccion = x.prod(p, '*', '*')

# costos fijos de produccion
c_fijo = y.prod(c, '*')

# costos de almacenamiento
almacenamiento = quicksum([h[t]*z[i,t] for i in I for t in T])

m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:
1. Balance entre produccion, almacenamiento y demanda.

In [5]:
# Balance primer periodo
m.addConstrs((x[i,1] - z[i,1] == d[i,1] for i in I), "balance_ini")

# Balance periodos siguientes
m.addConstrs((x[i,t] + z[i,t-1] - z[i,t] == d[i,t] for i in I for t in T2), "balance")

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update

2. Solamente puede producirse cuando $y_t =1$, y respetando la capacidad de la planta.

In [6]:
# y[t]=0 => sum_i (x[i, t])=0
m.addConstrs((x.sum('*', t) <= C*y[t] for t in T), "capacidad")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>}

Optimizar el modelo:

In [7]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 36 rows, 60 columns and 106 nonzeros
Model fingerprint: 0xc3dc15af
Variable types: 0 continuous, 60 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Found heuristic solution: objective 24974.000000
Presolve removed 22 rows and 40 columns
Presolve time: 0.00s
Presolved: 14 rows, 20 columns, 37 nonzeros
Found heuristic solution: objective 24971.000000
Variable types: 0 continuous, 20 integer (2 binary)

Root relaxation: objective 2.494900e+04, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    24949.000000 24949.0000  0.00%     -    0s

Explored 0 nodes (11 simplex iterations) in 0.20 seconds
Thread count was 4 (of 4 availabl

Mostrar la solución: producción, demanda y almacenamiento de cada producto en cada período.

In [8]:
# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
for i in I:
    print('Producto {}'.format(i))
    print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
    for t in T:
        print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[i,t], vy[t], vz[i,t], d[i,t]))

Producto 1
t		x_t		y_t		z_t		d_t
1		20.0		1.0		0.0		20
2		20.0		1.0		-0.0		20
3		26.0		1.0		1.0		25
4		24.0		1.0		-0.0		25
5		25.0		1.0		-0.0		25
6		20.0		1.0		-0.0		20
7		20.0		1.0		-0.0		20
8		18.0		1.0		-0.0		18
9		19.0		1.0		1.0		18
10		26.0		1.0		7.0		20
11		25.0		1.0		7.0		25
12		20.0		1.0		0.0		27
Producto 2
t		x_t		y_t		z_t		d_t
1		15.0		1.0		0.0		15
2		15.0		1.0		1.0		14
3		14.0		1.0		-0.0		15
4		16.0		1.0		-0.0		16
5		15.0		1.0		-0.0		15
6		17.0		1.0		-0.0		17
7		18.0		1.0		-0.0		18
8		17.0		1.0		-0.0		17
9		15.0		1.0		-0.0		15
10		14.0		1.0		-0.0		14
11		15.0		1.0		0.0		15
12		20.0		1.0		0.0		20


## Bonus extra
### Exportar el programa lineal entero a un archivo

A menudo es útil exportar el modelo de programación lineal entera a un archivo, con el objetivo de depurarlo, o de intentar resolverlo en otro solver. En la interfaz Python de Gurobi, esto puede hacerse llamando al método `write` de la clase modelo, cuyo argumento indica el nombre del archivo. La extensión indica el formato a utilizar en la exportación, un formato común es `.lp`. Más detalles acerca de este método pueden ser consultados en el [manual de referencia de Gurobi.](https://www.gurobi.com/documentation/9.0/refman/py_model_write.html)

In [10]:
m.write('lot-sizing-2.lp')

### Escribir tablas en Python
Existen algunos módulos de Python que facilitan la escritura de tablas en la consola. Uno de ellos es `tabulate`. Para instalar este módulo (si no está disponible aún en el sistema) se usa el comando `pip install`. Luego de ello será necesario **reiniciar el kernel** de este cuaderno.

In [12]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


Una vez instalado el módulo, podemos hacer uso de la función `tabulate` dentro del mismo, en combinación con el comando `print`. De esta manera, podemos escribir una lista bidimensional (lista de listas) como una tabla de manera sencilla. La función admite parámetros adicionales para formatear la escritura, uno de ellos es `headers`, que permite especificar los encabezados de las columnas. Otros detalles pueden ser consultados en [esta página web.](https://pypi.org/project/tabulate/)

In [11]:
from tabulate import tabulate

# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
for i in I:
    print('Producto {}'.format(i))
    tabla = [[t, vx[i,t], vy[t], vz[i,t], d[i,t]] for t in T]
    # print(tabla)
    # print(tabulate(tabla))
    print(tabulate(tabla, headers=['t', 'x_t', 'y_t', 'z_t', 'd_t']))

Producto 1
  t    x_t    y_t    z_t    d_t
---  -----  -----  -----  -----
  1     20      1      0     20
  2     20      1     -0     20
  3     26      1      1     25
  4     24      1     -0     25
  5     25      1     -0     25
  6     20      1     -0     20
  7     20      1     -0     20
  8     18      1     -0     18
  9     19      1      1     18
 10     26      1      7     20
 11     25      1      7     25
 12     20      1      0     27
Producto 2
  t    x_t    y_t    z_t    d_t
---  -----  -----  -----  -----
  1     15      1      0     15
  2     15      1      1     14
  3     14      1     -0     15
  4     16      1     -0     16
  5     15      1     -0     15
  6     17      1     -0     17
  7     18      1     -0     18
  8     17      1     -0     17
  9     15      1     -0     15
 10     14      1     -0     14
 11     15      1      0     15
 12     20      1      0     20


## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [9]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de planificación de la producción con varios productos y capacidad limitada de producción
# EPN (2020)

from gurobipy import *
try:
    # Conjuntos y parametros del modelo
    # Costos fijos y costos de almacenamiento por período:
    T, c, h = multidict({
        1 : (400, 10),   2 : (450, 12),  3 : (400, 10),
        4 : (470,  9),   5 : (450, 12),  6 : (400, 10),
        7 : (350, 10),   8 : (400, 12),  9 : (450, 10),
       10 : (450, 12),  11 : (500, 15), 12 : (500,  5)})


    # Costos de producción y demanda por (producto-período):
    IxT, p, d = multidict({
        (1, 1) : (48, 20),   (1, 2) : (47, 20),  (1, 3) : (38, 25),
        (1, 4) : (45, 25),   (1, 5) : (47, 25),  (1, 6) : (42, 20),
        (1, 7) : (33, 20),   (1, 8) : (39, 18),  (1, 9) : (45, 18),
        (1,10) : (47, 20),   (1,11) : (50, 25),  (1,12) : (55, 27),
        (2, 1) : (44, 15),   (2, 2) : (42, 14),  (2, 3) : (35, 15),
        (2, 4) : (41, 16),   (2, 5) : (42, 15),  (2, 6) : (38, 17),
        (2, 7) : (33, 18),   (2, 8) : (36, 17),  (2, 9) : (41, 15),
        (2,10) : (42, 14),   (2,11) : (45, 15),  (2,12) : (47, 20)})

    # Capacidad de la planta (lotes por período)
    C = 40

    # Recuperar conjunto I de productos desde IxT
    I = tuplelist(set([i for (i,t) in IxT]))

    # Definir conjunto T \ {1}
    T2 = [t for t in T if not t==1]
    
    # Crear el objeto modelo
    m = Model('lot-sizing-2')

    # Variables de decision
    # lotes a producir
    x = m.addVars(I, T, vtype = GRB.INTEGER, name="x")

    # se produce / no se produce en cada periodo
    y = m.addVars(T, vtype = GRB.BINARY, name="y")

    # lotes a almacenar
    z = m.addVars(I, T, vtype = GRB.INTEGER, name="z")    

    # Funcion objetivo
    # costos variables de produccion
    c_produccion = x.prod(p, '*')

    # costos fijos de produccion
    c_fijo = y.prod(c, '*')

    # costos de almacenamiento
    almacenamiento = quicksum([h[t]*z[i,t] for i in I for t in T])

    m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

    # Restricciones
    # Balance primer periodo
    m.addConstrs((x[i,1] - z[i,1] == d[i,1] for i in I), "balance_ini")

    # Balance periodos siguientes
    m.addConstrs((x[i,t] + z[i,t-1] - z[i,t] == d[i,t] for i in I for t in T2), "balance")   
    
    # y[t]=0 => sum_i (x[i, t])=0
    m.addConstrs((x.sum('*', t) <= C*y[t] for t in T), "capacidad")   
    
    # Resolver el modelo
    m.optimize()
    
    # Mostrar solucion
    # Extraer valores de las soluciones
    vx = m.getAttr('x', x)
    vy = m.getAttr('x', y)
    vz = m.getAttr('x', z)

    # Mostrar tabla con los valores
    for i in I:
        print('Producto {}'.format(i))
        print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
        for t in T:
            print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[i,t], vy[t], vz[i,t], d[i,t]))  
            
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 36 rows, 60 columns and 106 nonzeros
Model fingerprint: 0xc3dc15af
Variable types: 0 continuous, 60 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Found heuristic solution: objective 24974.000000
Presolve removed 22 rows and 40 columns
Presolve time: 0.00s
Presolved: 14 rows, 20 columns, 37 nonzeros
Found heuristic solution: objective 24971.000000
Variable types: 0 continuous, 20 integer (2 binary)

Root relaxation: objective 2.494900e+04, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    24949.000000 24949.0000  0.00%     -    0s

Explored 0 nodes (11 simplex iterations) in 0.16 seconds
Thread count was 4 (of 4 availabl